In [1]:
import pickle, os
with open("/home/zhaoheng_huang/CAGS_data/Pretrain/data/aol/train/id2sent.pkl", "rb") as fp:
    id2sent = pickle.load(fp)
with open("/home/zhaoheng_huang/CAGS_data/Pretrain/data/aol/train/sent2id.pkl", "rb") as fp:
    sent2id = pickle.load(fp)
res_path = "/home/zhaoheng_huang/CAGS_data/preprocess"
traindata_path = "/home/zhaoheng_huang/CAGS_data/Rank/data/aol/train/train_line.txt"

In [2]:
with open(os.path.join(res_path, "utterdict.pkl"), 'wb') as fp:
    """
    utterdict: int -> utter 或者utter -> int
    """
    res = {}
    for i, sent in id2sent.items():
        res[i] = sent
    for sent, i in sent2id.items():
        res[sent] = i
    pickle.dump(res, fp)

In [3]:
class Utterdict():
    def __init__(self, path: str) -> None:
        with open(path, "rb") as fp:
            self.utterdict = pickle.load(fp)
        self.querydict, self.querydictid = self.extractQ()
        self.docdict, self.docdictid = self.extractD()
    def extractQ(self) -> dict:
        res = {}
        resid = []
        for k, v in self.utterdict.items():
            if type(v) == str and v.startswith("<q>"):
                res[k] = v[3:]
                res[v[3:]] = k
                resid.append(k)
        return res, resid
    def extractD(self) -> dict:
        res = {}
        resid = []
        for k, v in self.utterdict.items():
            if type(v) == str and v.startswith("<d>"):
                res[k] = v[3:]
                res[v[3:]] = k
                resid.append(k)
        return res, resid

In [4]:
utter = Utterdict(os.path.join(res_path, "utterdict.pkl"))

In [5]:
with open("/home/zhaoheng_huang/CAGS_data/Rank/data/aol/train/train_line.txt", "r") as fp:
    sessList = fp.readlines()
    def wash(x) -> list:
        x = x.strip().split("\t")
        for i in range(len(x)):
            x[i] = x[i].strip()
        return x
    sessList = list(map(wash, sessList))

In [6]:
from collections import defaultdict
clickGraph = defaultdict(set)  # q -> d, d -> q
for sess in sessList:
    if sess[0] == '1':  # 点击
        sess = sess[1:]
        for i in range(0, len(sess), 2):
            q, d = sess[i], sess[i+1]
            q_id, d_id = utter.querydict[q], utter.docdict[d]
            clickGraph[q_id].add(d_id)
            clickGraph[d_id].add(q_id)

In [18]:
with open("/home/zhaoheng_huang/CAGS_data/TransE/train.txt", "w") as fp:
    for node in clickGraph:
        outernodes = list(clickGraph[node])
        for tail in outernodes: # node, tail为一对边
            fp.write("\t".join([str(node), "0", str(tail)]) + "\n")

In [19]:
!head /home/zhaoheng_huang/CAGS_data/TransE/train.txt

125696	0	319691
125696	0	272989
319691	0	151554
319691	0	81925
319691	0	81926
319691	0	81929
319691	0	125453
319691	0	98849
319691	0	98850
319691	0	98851
